# Using ML Croissant with FAID

ML Croissant is a 

We can also use directly its RAI specification, if the author of the dataset entered the details.
https://docs.mlcommons.org/croissant/docs/croissant-rai-spec.html

On their web page, the authors specify the the use cases of Croissant's metadata in RAI as follows:

| RAI use case                         | Croissant RAI properties                                                                 | Croissant properties                                                                                                                                                 | Schema.org properties                                                                                                           |
|--------------------------------------|------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------|
| Use case 1: The data life cycle      | rai:dataLimitations<br>rai:dataCollection<br>rai:useCases<br>rai:dataReleaseMaintenance   | cr:[distribution](http://schema.org/distribution)<br>cr:[isLiveDataset](https://mlcommons.org/croissant/1.0#modified-and-added-properties)<br>cr:[citeAs](https://mlcommons.org/croissant/1.0#modified-and-added-properties)    | sc:[creator](http://schema.org/creator)<br>sc:[publisher](http://schema.org/publisher)<br>sc:[datePublished](http://schema.org/datePublished)<br>sc:[dateCreated](http://schema.org/dateCreated)<br>sc:[dateModified](http://schema.org/dateModified)<br>sc:[version](http://schema.org/version)<br>sc:[license](http://schema.org/license)<br>sc:[maintainer](https://schema.org/maintainer) |
| Use case 2: Data labeling            | rai:annotationPlatform<br>rai:annotationsPerItem<br>rai:annotatorDemographics<br>rai:machineAnnotationTools   | cr:[distribution](http://schema.org/distribution)<br>cr:[isLiveDataset](https://mlcommons.org/croissant/1.0#modified-and-added-properties)<br>[cr:Label](https://mlcommons.org/croissant/1.0#label-data)       |                                                                                                                               |
| Use case 3: Participatory data       | rai:annotationPlatform<br>rai:annotatorDemographics                                       |                                                                                                                                                                      | sc:[participant](https://schema.org/participant)<br>sc:[contributor](https://schema.org/contributor)                         |
| Use case 4: AI safety and fairness   | rai:dataLimitations<br>rai:dataBiases<br>rai:useCases<br>rai:personalSensitiveInformation |                                                                                                                                                                      | sc:[diversityPolicy](https://schema.org/diversityPolicy)<br>sc:[ethicsPolicy](https://schema.org/ethicsPolicy)<br>sc:[inLanguage](https://schema.org/inLanguage) |
| Use case 5: Traceability             |                                                                                          |                                                                                                                                                                      |                                                                                                                               |
| Use case 6: Regulatory compliance    | rai:personalSensitiveInformation<br>rai:useCases<br>rai:dataReleaseMaintenance<br>rai:dataManipulationProtocol<br>rai:dataManipulationProtocol |                                                                                                                                                                      |                                                                                                                               |
| Use case 7: Inclusion                |                                                                                          |                                                                                                                                                                      |                                                                                                                               |



In [1]:
from mlcroissant import Dataset

ds = Dataset(jsonld="https://huggingface.co/api/datasets/HannahRoseKirk/prism-alignment/croissant")
records = ds.records("conversations")

  -  [Metadata(prism-alignment)] Property "http://mlcommons.org/croissant/citeAs" is recommended, but does not exist.
  -  [Metadata(prism-alignment)] Property "https://schema.org/datePublished" is recommended, but does not exist.
  -  [Metadata(prism-alignment)] Property "https://schema.org/version" is recommended, but does not exist.


In [2]:
meta = ds.metadata.to_json()

In [3]:
meta.keys()

dict_keys(['@context', '@type', 'name', 'description', 'conformsTo', 'creator', 'keywords', 'license', 'url', 'distribution', 'recordSet'])

In [4]:
meta["conformsTo"] # If dataset conforms to "http://mlcommons.org/croissant/RAI/1.0" it includes RAI properties

'http://mlcommons.org/croissant/1.0'

In [5]:
print(meta["name"])
print(meta["description"].replace("\n", "-").replace("\t", ""))
print(meta["license"])

prism-alignment
-----Dataset Card for PRISM---PRISM is a diverse human feedback dataset for preference and value alignment in Large Language Models (LLMs).-It maps the characteristics and stated preferences of humans from a detailed survey onto their real-time interactions with LLMs and contextual preference ratings------Dataset Details---There are two sequential stages: first, participants complete a Survey where they answer questions about their demographics and stated preferences, then… See the full description on the dataset page: https://huggingface.co/datasets/HannahRoseKirk/prism-alignment.
https://choosealicense.com/licenses/cc/


In [6]:
# You can even directly call rows directly from huggingface using the croissant metadata information
import itertools
import pandas as pd

df = pd.DataFrame(list(itertools.islice(records, 10)))
df.head()

,conversations/conversation_id,conversations/user_id,conversations/conversation_type,conversations/opening_prompt,conversations/conversation_turns,conversations/open_feedback,conversations/timing_duration_s,conversations/timing_duration_mins,conversations/included_in_balanced_subset
0,b'c0',b'user0',b'unguided',b'What can you do about the inequality of wea...,5,b'Shorter blocks would be nice. but has to ...,401.07,6.68,False
1,b'c1',b'user1',b'unguided',b'What can I do to start making extra money on...,2,b'The first time I responded to the second que...,642.97,10.72,True
2,b'c2',b'user4',b'controversy guided',b'Who is right in the Hamas-Israeli war? Hama...,2,b'The AI stance is taking more of a neutral st...,362.35,6.04,True
3,b'c3',b'user6',b'unguided',b'Hello',2,b'It would make thing easier if the AI include...,393.35,6.56,True
4,b'c4',b'user2',b'unguided',b'How do I become financially stable on a low ...,2,b'I had a good conversation overall. It would ...,765.24,12.75,True


In [7]:
from faid.faidlog import faidlog
# Generate a report-friendly summary of the dataset
dataset_info = faidlog.pretty_croissant(ds)
faidlog.log(dataset_info, "croissant_meta", add_to_data_card=True)
print(dataset_info)

Added croissant_meta to data card
{'dataset_name': 'prism-alignment', 'summary': '\n\t\n\t\t\n\t\n\t\n\t\tDataset Card for PRISM\n\t\n\nPRISM is a diverse human feedback dataset for preference and value alignment in Large Language Models (LLMs).\nIt maps the characteristics and stated preferences of humans from a detailed survey onto their real-time interactions with LLMs and contextual preference ratings\n\n\t\n\t\t\n\t\n\t\n\t\tDataset Details\n\t\n\nThere are two sequential stages: first, participants complete a Survey where they answer questions about their demographics and stated preferences, then… See the full description on the dataset page: https://huggingface.co/datasets/HannahRoseKirk/prism-alignment.', 'dataset_link': 'https://huggingface.co/datasets/HannahRoseKirk/prism-alignment', 'authors': '', 'publishing_organization': '', 'industry_types': [], 'publishing_poc': {}, 'dataset_owners': {'@type': 'sc:Person', 'name': 'Hannah Rose Kirk', 'url': 'https://huggingface.co/Hanna

In [8]:
from faid.report.report_utils import generate_data_card
generate_data_card(dataset_info, "prism_alignment_data_card.html")

## Include RAI Fields

An example the authors provided is Google's DICES dataset (https://github.com/google-research-datasets/dices-dataset/). However, the dataset does not include a croissant metadata online. It is just an example illustration:

In [9]:
rai_json  = {
  "@context": {
    "@language": "en",
    "rai": "http://mlcommons.org/croissant/RAI/",
    "sc": "https://schema.org/"
  },
  "@type": "schema.org/Dataset",
  "name": "DICES-30",
  "dct:conformsTo": "http://mlcommons.org/croissant/RAI/1.0",
  "rai:dataCollection": "The input data was sampled from an 8K multi-turn conversation corpus (comprising 48K turns in total) generated by human agents interacting with a generative AI-chatbot. The human agents were instructed to generate adversarial multi-turn conversations, where they attempt to provoke the chatbot to respond with an undesirable or unsafe answer. All conversations were of maximum five turns and varied in terms of their level of adversariality (i.e., degree of harm) and topics.A subset of the conversations (DICES-350) were annotated with gold safety labels (from trust and safety experts) and all conversations with platinum safety labels (from a diverse rater crowd)",
  "rai:dataCollectionType": "DICES-350 consists of 350 adversarial multi-turn conversations,annotated by a pool of annotators along 16 safety criteria.",
  "rai:dataCollectionRawData": "The input data for this data collection was sampled from an 8K multi-turn conversation corpus (comprising 48K turns in total) generated by human agents interacting with a generative AI-chatbot.",
  "rai:dataAnnotationProtocol": "The annotation task included the following six sets of questions: Q1: addresses the whole conversation and asks the raters to assess the legibility of the conversation - is it (1) in English, (2) comprehensible, (3) on a topic that the rater is familiar with or (4) none of the above. Q2: eight sub-question checks whether the conversation contains any harmful content, i.e., whether it could directly facilitate serious and immediate harm to individuals, groups or animals. Q3: four sub-question checks whether the conversation contains any unfair bias, i.e., whether it incites hatred against an individual or group. Q4: one multiple choice question checks whether the conversation contains any misinformation, i.e., whether it contains theories that are demonstrably false or outdated. Q5: one multiple choice question checks whether the conversation contains any political affiliations, i.e., engaging in or downplaying any controversial topic. Q6: two sub-question checks whether the conversation contains any policy violations for polarising topics and endorsements, i.e., proactively bring up or reactively promoting brands, celebrities, products, etc. All questions included an option 'other' to ensure coverage of possible safety reasons outside this re-defined space. Rating options for Q2 to Q6 were: “No” (i.e., safe), “Yes” (i.e., unsafe) and “Unsure”. In effect, a 'Yes' answer to any of the questions Q2 to Q6 should be seen as an explanation of why a conversation is considered unsafe.",
  "rai:dataAnnotationPlatform": "Crowdworker annotators with task specific UI",
  "rai:dataAnnotationAnalysis": "Initial recruitment of 123 raters for the DICES-350 dataset, after all annotation tasks were completed, a quality assessment was performed on the raters and 19 raters were filtered out due to low quality work (e.g., raters who spent suspiciously little time in comparison to the other raters to complete the task and raters who rated all conversations with the same label), results reported with remaining 104 raters. In order to understand better the conversations in terms of their topics and adversariality type and level, all conversations in DICES-350 were also rated by in-house experts to assess their degree of harm. All conversations in DICES-350 have gold ratings,i.e. they were annotated for safety by a trust and safety expert. Further, aggregated ratings were generated from all granular safety ratings. They include a single aggregated overall safety rating ('Q_overall'), and aggregated ratings for the three safety categories that the 16 more granular safety ratings correspond to: 'Harmful content' ('Q2_harmful_content_overall'), 'Unfair bias' ('Q3_bias_overall') and 'Safety policy violations' ('Q6_policy_guidelines_overall').",
  "rai:dataUseCases": "The dataset is to be used as a shared resource and benchmark that respects diverse perspectives during safety evaluation of conversational AI systems.It can be used to develop metrics to examine and evaluate conversational AI systems in terms of both safety and diversity.",
  "rai:dataBiases": "Dataset includes multiple sub-ratings which specify the type of safety concern, such as type of hate speech and the type of bias or misinformation, for each conversation. A limitation of the dataset is the selection of demographic characteristics. The number of demographic categories was limited to four (race/ethnicity, gender and age group). Within these demographic axes, the number of subgroups was further limited (i.e., two locales, five main ethnicity groups, three age groups and two genders), this constrained the insights from systematic differences between different groupings of raters.",
  "rai:annotationsPerItem": "350 conversations were rated along 16 safety criteria, i.e.,104 unique ratings per conversation.",
  "rai:annotatorDemographics": "DICES-350 was annotated by a pool of 104 raters. The rater breakdown for this pool is: 57 women and 47 men; 27 gen X+, 28 millennial, and 49 gen z; and 21 Asian, 23 Black/African American, 22 Latine/x, 13 multiracial and 25 white. All raters signed a consent form agreeing for the detailed demographics to be collected for this task."
}

In [ ]:
# Add this JSON to the dataset metadata
faidlog.log(rai_json, "croissant_rai", add_to_data_card=True)

In [1]:
from faid.faidlog import faidlog

In [2]:
faidlog.generate_trust_label()

We found the keys: ['sample_results', 'variable_profile', 'bygroup_metrics']. Based on the log, the fairness log completeness is calculated: 1.0
